# *A.09 排序

### 9.1 众多排序算法

- 插入排序
- 选择排序
- 合并排序
- 快速排序
- 冒泡排序
- 其它许多

如下，用几行Python代码实现*选择排序*

In [ ]:
import numpy as np

In [ ]:
def selection_sort(x):
    for i in range(len(x)):
        swap = i + np.argmin(x[i:])
        (x[i], x[swap]) = (x[swap], x[i])
    return x

In [ ]:
x = np.array([2, 1, 4, 3, 5])
selection_sort(x)

选择排序适用于简单问题，对于大数组的运算速度太慢

对于 $N$ 个元素的列表，选择排序算法的计算量平均约为 $\mathcal{O}[N^2]$，如果元素增加一倍，计算时间将乘以4。

不管怎样，选择排序算法也远好于bogo排序方法。

In [ ]:
def bogosort(x):
    while np.any(x[:-1] > x[1:]):
        np.random.shuffle(x)
    return x

In [ ]:
x = np.array([2, 1, 4, 3, 5])
bogosort(x)

- 上述方法的平均计算量为 $\mathcal{O}[N \times N!]$
- 因此，这一排序方法从未被用于实际计算

### 9.2 NumPy 中的快速排序算法: ``np.sort`` 和 ``np.argsort``

- Python 有内建的``sort``和``sorted``函数用于对列表排序
- NumPy 也有效率更高的``np.sort``函数


缺省时，``np.sort``使用 $\mathcal{O}[N\log N]$阶的*快速排序算法*，尽管*合并排序法*和*堆排序法*也可用，对于绝大多数情形，缺省的快速排序算法完全够用了

返回排好序的结果，又不修改输入数组，可以使用``np.sort``

In [ ]:
x = np.array([2, 1, 4, 3, 5])
np.sort(x)

如果想就地（in-place）排序一数组，可采用数组的``sort``方法

In [ ]:
x.sort()
print(x)

相应的函数是``argsort``，它返回的是排好序的元素索引

In [ ]:
x = np.array([2, 1, 4, 3, 5])
i = np.argsort(x)
print(i)

返回的索引列表可用来（通过花式索引）构建排好序的数组

In [ ]:
x[i]

### 9.3 沿行或列排序

NumPy 可沿多维数组特定的行或列进行排序，这要配合使用``axis``参数，例如

In [ ]:
rand = np.random.RandomState(42)
X = rand.randint(0, 10, (4, 6))
print(X)

In [ ]:
# sort each column of X
np.sort(X, axis=0)

In [ ]:
# sort each row of X
np.sort(X, axis=1)

### 记住

- 上述操作<font color="red">将行或列当成独立数组</font>，而且原来行间或列间元素的关系全部失去

### 9.4 部分排序：分区(Partitioning)

有时，可能对数组整体排序没有兴趣，但是希望找到数组中*k*个最小的值，NumPy为此提供了``np.partition``函数。

- ``np.partition``函数的参数
    - 一个数组参数
    - 一个数*K*
    - 结果是一个新数组，含*K*个最小的值在分区的左边，余下的右边数组值，以任意序排列

In [ ]:
x = np.array([7, 2, 3, 1, 6, 5, 4])
np.partition(x, 3)

#### 注意到

- 结果数组中，最前面三个数是三个最小值
- 两个分区中，元素无序排列

类似于排序，也可沿多维数组的任意轴方向进行分区排序

In [ ]:
np.partition(X, 2, axis=1)

- 上述结果各行的前2个元素分别是各行最小的2个值
- 如同有``np.argsort``计算排序的索引，也有``np.argpartition``计算分区的索引

### 9.4 示例：k-最近邻(k-Nearest Neighbors)

关注：使用``argsort``函数沿多轴寻找集合中每点的最近邻居

先在二维平面内创建10个随机点

同常规，将这10个点放入 $10\times 2$ 数组

In [ ]:
X = rand.rand(10, 2)
X

- 想了解这些点长成什么样？
- 快速绘制一散点图

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set() # Plot styling
plt.scatter(X[:, 0], X[:, 1], s=100);

- 计算每对点间的距离
- 点距离的计算要用到勾股定理
    - 使用广播技术
    - 使用聚合函数
- 可以只用单行代码完成上述计算

In [ ]:
dist_sq = np.sum((X[:, np.newaxis, :] - X[np.newaxis, :, :]) ** 2, axis=-1)

- 上述操作十分紧凑，如果不熟悉NumPy的广播规则，有可能会感到困惑
- 如果遇到上述代码，也可不采用紧凑代码，而是分步骤计算

In [ ]:
# for each pair of points, compute differences in their coordinates
differences = X[:, np.newaxis, :] - X[np.newaxis, :, :]
differences.shape

In [ ]:
# square the coordinate differences
sq_differences = differences ** 2
sq_differences.shape

In [ ]:
# sum the coordinate differences to get the squared distance
dist_sq = sq_differences.sum(-1)
dist_sq.shape

#### 进行复查

- 矩阵对角元素应该全为零（点到自身的距离）

In [ ]:
dist_sq.diagonal()

检查后！

- 使用``np.argsort``对每行排序
- 最左边的列将给出最近邻的索引

In [ ]:
nearest = np.argsort(dist_sq, axis=1)
print(nearest)

#### 注意到

- 第一列依次给出了0到9，这是因为每个点最近的邻居是自己
- 如果使用完全排序，我们就做了多余的工作
- 如果只是对最近的 $k$ 邻居有兴趣，需要做的是对每行分区，并得到最小的 $k+1$ 个平方距离
- 使用``np.argpartition``函数

In [ ]:
K = 2
nearest_partition = np.argpartition(dist_sq, K + 1, axis=1)

#### 可视化邻居网络

- 画出所有点
- 画出各点最近的2个邻居及连线

In [ ]:
plt.scatter(X[:, 0], X[:, 1], s=100)

# draw lines from each point to its two nearest neighbors
K = 2

for i in range(X.shape[0]):
    for j in nearest_partition[i, :K+1]:
        # plot a line from X[i] to X[j]
        # use some zip magic to make it happen:
        plt.plot(*zip(X[j], X[i]), color='black')

#### 观察上图

- 每个点都有若干条线连到它最近的邻居

初看发现

- 有些点的连线不止两条，这是合理的，因为即使点A是点B的近邻，并不一定点B是点A的近邻

广播和行排序与循环相比，看起来并不直观，但却是非常有效率的操作方式

如果进行人工循环，将会降低算法的效率

最后，如果进行非常大量的最近邻居搜索，有3种逼近方法，能达到$\mathcal{O}[N\log N]$甚至更好的阶次，而不是brute-force算法的$\mathcal{O}[N^2]$阶，例如 KD-Tree，在[Scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html)中实现。

### 结束